Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Visualization - Grouping and Appearance Customization

In this notebook we'll examine how to use the `--group-by` feature of the `%%gremlin` magic to group vertices and how to customize the appearance of these groups. 

In addition, we will look at how to use the `--display-property`, `--edge-display-property`, and `--label-max-length` options to modify the appearances of node and edge labels.

**Note** This notebook builds on the visualization patterns explained in [Air-Routes-Gremlin](Air-Routes-Gremlin.ipynb) notebook, so if you are not familiar with how the visualzation feaure works we recommend reviewing that prior to beginning this notebook.

This notebook uses the air-routes dataset to demonstrate some of the different options available.  If your cluster does not already contain this data then run the cell below to load the data into your Neptune cluster otherwise you can skip the next cell.

In [ ]:
%seed --language Gremlin --dataset airports --run

With the air-routes data now loaded we are ready to begin looking at how to group and customize our result visualization.  

## Node Property Options

Visualizing the results of a query will result in a graph where each vertex contains an identifying property. By default, the property used is generated automatically from the label property. If desired, it can instead abide by the property (or set of label properties) specified using the `--display-property` or `-d` parameter, followed by the property name. 

Additionally, labels are truncated after exceeding a default maximum length. This maximum length value can be modified by using the `--label-max-length` or `-l` parameter, followed by the desired length in characters.


### Default Node Properties

By default, the property used is the value of each vertex's label property. Run the query below to observe the default labeling for results set of all vertices connected to Cozumel.

In [ ]:
%%gremlin -p v,inv
g.V().hasLabel('airport').has('code','CZM').both().path()

The results show us only three distinct labels, corresponding to each of the label properties `airport`, `country`, and `continent`.

In some cases, a label property may not present may not be present in some or all of the vertices returned in the results set (i.e. T.label has not been returned in the results via the appropriate valueMap() step). 

This will result in a concatenated list of all of the vertex's properties being displayed instead, as we can observe with the following query.

In [ ]:
%%gremlin -p v,inv
g.V().hasLabel('airport').has('code','CZM').both().path().by(valueMap())

### Specifying a Single Node Property for all Vertices

There may also be cases where it is desired to show a specific vertex property as the label on each graph node. This can be done by using the `--display-property` or `-d` parameter within the `%%gremlin` line magic. The property name needs to be a case sensitive match for the name in the vertex.  

**Note** Finding the property name can be accomplished using the Details View and clicking on a vertex.  This includes the `T.id` and `T.label` properties.

**Note** We must use the `valueMap(true)` step to ensure that the results the label names and relevant properties for each vertex.

Let's run the following query to see the results of displaying the results set of all vertices connected to Cozumel, while also specifying that we want to display the `code` property on every vertex.

In [ ]:
%%gremlin -p v,inv -d code
g.V().hasLabel('airport').has('code','CZM').both().path().by(valueMap(true))

Looking at the resulting visualized graph, each individual node can now be identified by its distinct code.

### Using Different Node Properties for each Label

Instead of displaying the values of a single property, we can also choose to specify different properties to display for each type of label. This feature can be useful if different labels in the graph have different property sets, or if you want only a subset of vertices under certain labels to have the displayed property modified.

We will first need to define a JSON-format string variable in the following format, containing each label and its corresponding propreties to be displayed:

`display_var = '{"label_1":"property_1","label_2":"property_2"}'`

Let's try using this to define different display properties for the `airport`, `country`, and `continent` labels.

In [ ]:
display_var = '{"airport":"code","country":"desc","continent":"desc"}'

Now, we can take the previous query and pass `display_var` into the displayed properties parameter via the notebooks line variable injection functionality.

In [ ]:
%%gremlin -p v,inv -d $display_var
g.V().hasLabel('airport').has('code','CZM').both().path().by(valueMap(true))

In the resulting visualization, we can see that the `airport` vertices retain the `code` property values, while the `country` and `continent` nodes now display their more descriptive `desc` property values, as we desired.

### Specifying the Maximum Label Length

All node labels in the graph abide by a maximum length value, after which they are visually truncated. By default, this value is 10.

You may encounter scenarios where you want to modify the truncation length to either be shorter or longer than the default value. This can be done by adding the `--label-max-length` or `-l` parameter to your Gremlin query, followed by the desired length in characters.

Let's try to visualize a graph of all airports connected to Cozumel, while indicating that we want to display the full airport names on the nodes. Note that the label truncation length is left as default.

In [ ]:
%%gremlin -p v,inv -d desc
g.V().hasLabel('airport').has('code','CZM').both().path().by(valueMap(true))

We can see that most of the labels are cut off too early to be adequately descriptive!

Let's try the same query again, this time incorporating the `-l` parameter, followed by a maximum length value of `60`.

In [ ]:
%%gremlin -p v,inv -d desc -l 60
g.V().hasLabel('airport').has('code','CZM').both().path().by(valueMap(true))

Now, we can see the full name of every airport on the graph.

## Edge Property Options

Some graph visualizations of Gremlin magic queries will contain edges that are also accompanied by a visual label. The property displayed on each edge defaults to the edge's label property. 

We can specify what property values to display on the visualized edges in a similar fashion to how we customized node labels in the previous section. This is done by using the `--edge-display-property` or `-de` parameter, followed by the property name, or a json formatted string containing label-value:property-name pairs. 

### Specifying a single property for all edge labels

Using a single property name as the parameter value for `--edge-display-property` will result every edge label being changed to property. Again, note that you can find the properties available by using the details view and clicking on any edge in the graph visualization.

Let's observe the results of appending `-de outV` to a query that retrieves all routes from Austin, USA to Wellington, NZ. Note that we forgo usage of any `by` modulators in the query to ensure that we are retrieving all of the edge properties.

In [ ]:
%%gremlin -p v,oute,inv,oute,inv,oute,inv -de outV
g.V().has('airport','code','AUS').
  repeat(outE().inV().simplePath()).
  until(has('code','WLG')).
  limit(5).
  path()

We can see that every edge is now indicated by its outgoing vertex via the `outV` property.

### Specifying edge label properties in JSON format

Making use of the cell variable injection feature, we can also define a JSON-format string variable containing our desired label-value:property-name pairs, and pass it into `-de`. This follows the format used when injecting JSON variables into `-d` and `-g`:

`edge_display_var = '{"label_1":"property_1","label_2":"property_2"}'`

Let's re-define the same label-value:property-name pair used in the last query, this time in the JSON string variable format.

In [ ]:
display_edge_var = '{"route":"inV"}'

Then, we can pass `$display_edge_var` into `-de`, and observe that we get the same result.

In [ ]:
%%gremlin -p v,oute,inv,oute,inv,oute,inv -de $display_edge_var
g.V().has('airport','code','AUS').
  repeat(outE().inV().simplePath()).
  until(has('code','WLG')).
  limit(5).
  path()

## Grouping Options

When visualizing the results of a query the grouping of the vertices follows a couple of rules:

* If no grouping property is specified and the label for a vertex is returned then the vertices are grouped by label.
    * If the label does not exist in the results then all vertices are grouped into a single group
* To group vertices by a specific property use the `--group-by` or `-g` switch on the `%%gremlin` line magic.  
    * The name provided must be a case sensitive match to the property name of the vertex (i.e. `Name` will not group by the `name` property)
    * If the name specified does not match any property of a vertex then that vertex is grouped in a default group
* If you would like to not group by any property then the `--ignore-groups` flag will not group the vertices

### Default Grouping

Let's take a look at what our air-routes graph looks like if we run a query and use the default grouping. Try running the query below and clicking on the Graph tab to see all the vertices connected to Cozumel.

In [ ]:
%%gremlin

g.V().has('airport','code','CZM').both().path()

From the results we see three groups each represented by a different color.  Each of the vertices is added to a group based on it's label.  

Grouping also occurs when we use the `by()` modulators to specify additional return fields.  Running the query below to see how the items are grouped when we find all connected vertices for Cozumel and return all their properties and tokens (T.id and T.label).

In [ ]:
%%gremlin

g.V().has('code', 'ANC').both().path().by(valueMap().with(WithOptions.tokens))

From the resultant visualization we see that we still have three groups of vertices, grouped by the label.  

However, what would happen if we don't return the label in the results.  Run the query below to see how the items are grouped when we find all connected vertices for Cozumel and return just their properties, not the label.

In [ ]:
%%gremlin

g.V().has('code', 'ANC').both().path().by(valueMap())

From the results we see that all our vertices are in a single group now and colored the same.  When our results do not contain the label for the vertex we must then specify the property we want to group by.  

### Specifying the property to group by

To specify the property to group by we use either the `--group-by` or `-g` switch followed by the property name.  The property name needs to be a case sensitive match for the name in the vertex.  

**Note** Finding the property name can be accomplished using the Details View and clicking on a vertex.  This includes the `T.id` and `T.label` properties.

Let's run the query below and see what our grouping looks like if we find all routes from Cozumel and group them by their `country`.

In [ ]:
%%gremlin -p v,inv -g country
g.V().hasLabel('airport').has('code','CZM').out('route').path().by(valueMap())

As we see our results are now split into three groups (MX/US/CA) based on the country property.  

Let's run the query below and see what our grouping looks like if we find all connections to Cozumel and group them by their `country`.

In [ ]:
%%gremlin -p v,inv -g country
g.V().hasLabel('airport').has('code','CZM').both().path().by(valueMap())

The resultant graph looks very similar to the previous one except that their is now four groups instead of three.  Unlike in the previous example where we only returned `route` connections, this query returned all connected vertices so our resultset came back with a `continent` and `country` vertex in addition to the `airport` vertices.  Neither the `country` nor `continent` vertices have a `country` property.  When a vertex does not contain the property specified to group by then that vertex is put into a default group.  This same behavior occurs when you use the incorrect casing for the property name, as shown in the query below.

In [ ]:
%%gremlin -p v,inv -g Country
g.V().hasLabel('airport').has('code','CZM').out('route').path().by(valueMap())

### Grouping on different properties for each label

While the `continent` and `country` vertices cannot be grouped on properties exclusive to `airport` vertices, we do have the ability to add additional properties with which to form new groups. To do this, we will make use of Jupyter's built-in line magic variable injection to pass in a variable containing a JSON-format string, where we can specify what property we want to use to group each of the individual vertex labels.  

The group-by variable must be defined in following format:  

`groups_var = '{"label_1":"property_1","label_2":"property_2"}'`

Let's try defining group-by with individual properties for `airport`, `continent`, and `country`.

In [ ]:
groups_var = '{"airport":"country","country":"desc","continent":"code"}'

Then, we can rerun our previous query, this time with `$groups_var`, and see that the `continent` and `country` vertices now belong to their own groups based on the specified properties.  

**Note** we also need to use `valueMap(true)` so that the label names and relevant properties can be accessed for group matching.

In [ ]:
%%gremlin -p v,inv -g $groups_var
g.V().hasLabel('airport').has('code','CZM').both().path().by(valueMap(true))

### Ignoring Grouping

In certain situations you may want your visualization to contain no groups.  This can be accomplished by adding the `--ignore-groups` flag to your query as shown by running the next cell.

In [ ]:
%%gremlin -p v,inv --ignore-groups
g.V().hasLabel('airport').has('code','CZM').out('route').path().by(valueMap())

Now that we know how to group our vertices together let's take a look at how to customize the appearance of these groups.

## Group Customization
The Amazon Neptune Notebooks use an open source library called [Vis.js](https://github.com/visjs) to assist with drawing the graph diagrams. Vis.js provides a rich set of customizable settings. The documentation for most of the visualization settings used in this notebook can be found [here](https://visjs.org/) and in particular the graph network drawing documentation can be found [here](https://visjs.github.io/vis-network/docs/network/).  

To see the current settings used by your notebook you can use the `%graph_notebook_vis_options` line magic command. Try running the cell below.  

To change any of these settings create a new cell and use `%%graph_notebook_vis_options` to change them (note the two percent signs indicating a cell magic).

To customize the appearance of the the groups we want to use the [groups](https://visjs.github.io/vis-network/docs/network/groups.html#) options.  Their is a nearly endless amount of customization you can make to the groups using the options provided but we will demonstrate some of the most common ones in the next few sections.

### Specifying Colors

Specifying the colors of groups is probably one of the most common customizations performed.  To accomplish this we specify the options using the `%%graph_notebook_vis_options` magic as shown below.  For each of the associated group names we use the exact property value followed by the options you would like to use for that group.

**Note** Finding the exact property value for the group name can be accomplished by looking at the data returned in the Console tab.

Run the next two cells to set the colors for our three groups to red for the airports in Canada, green for the airports in the US, and blue for the airports in Mexico.  In the case of color, the values can be specified by name, RGBA value, or Hex value.

In [ ]:
%%graph_notebook_vis_options
{
  "groups": {
      "['CA']": {"color": "red"},
      "['MX']": {"color": "rgba(9, 104, 178, 1)"},      
      "['US']": {"color": "#00FF00"}
    }
}

In [ ]:
%%gremlin -p v,inv -g country
g.V().hasLabel('airport').has('code','CZM').out('route').path().by(valueMap())

### Specifying Shape

In addition to specifying the color of the groups you are also able to customize the shape of the vertex using one of the following options. 

The types with the label inside of it are: `ellipse`, `circle`, `database`, `box`, `text`.

The ones with the label outside of it are: `diamond`, `dot`, `star`, `triangle`, `triangleDown`, `hexagon`, and `square`

Run the cells below to see what our visualization looks like with shapes specified.

In [ ]:
%%graph_notebook_vis_options
{
  "groups": {
      "['CA']": {"color": "red", "shape": "box"},
      "['MX']": {"color": "rgba(9, 104, 178, 1)" , "shape": "oval"},      
      "['US']": {"color": "#00FF00", "shape": "star"}
    }
}

In [ ]:
%%gremlin -p v,inv -g country
g.V().hasLabel('airport').has('code','CZM').out('route').path().by(valueMap())

### Specifying Icons and Images

In addition to specifying shapes icons and images can also be specified to represent our groups.  

Icons can either be from `Ionicons` or `FontAwesome` version 4 or 5.  Icons are specified by setting the `shape` to `icon` and then specifyign the `code` for the icon.

Images must be either local to the notebook or publically available on the internet.  Images are specified by setting the `shape` to `image` or `circularImage` and then setting the `image` property to the address of the image to display.

Running the two cells below will display the Mexico group using an image of the Mexican flag and the US group using an airplane icon from FontAwesome.

In [ ]:
%%graph_notebook_vis_options
{
  "groups": {
      "['CA']": {"color": "red"},
    "['MX']": {"shape": "image", 
               "image":"https://cdn.countryflags.com/thumbs/mexico/flag-round-250.png"},
    
    "['US']": {
      "shape": "icon",
      "icon": {
        "face": "FontAwesome",
        "code": "\uf072",
        "color": "#00FF00"
          }
      }
    }
}

In [ ]:
%%gremlin -p v,inv -g country
g.V().hasLabel('airport').has('code','CZM').out('route').path().by(valueMap())

### Specifying Size

The last customization we are going to demonstrate is the ability to set the size for the group.  This is accomplished by setting the `size` property of the group. 

**Note** Only shapes that do not have the label inside them are impacted by this property.

In [ ]:
%%graph_notebook_vis_options
{
  "groups": {
      "['CA']": {"color": "red", "size": 3},
    "['MX']": {"shape": "image", 
               "image":"https://cdn.countryflags.com/thumbs/mexico/flag-round-250.png"
              , "size": 50},
    
    "['US']": {
      "shape": "icon",
      "icon": {
        "face": "FontAwesome",
        "code": "\uf072",
        "color": "#00FF00"
          }, "size": 37
      }
    }
}

In [ ]:
%%gremlin -p v,inv -g country
g.V().hasLabel('airport').has('code','CZM').out('route').path().by(valueMap())

## Conclusion

What we have demonstrated in this notebook is only a small set of the options and combinations available for customizing the appearance of groups within the notebook.  Please refer to the [Vis.js groups](https://visjs.github.io/vis-network/docs/network/groups.html#) documentation for a complete list of the options available.